In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
from PIL import Image
import tensorflow as tf
import numpy as np
from scipy import spatial

In [ ]:
import keras
from keras.callbacks import Callback
import os

In [ ]:
from keras.models import load_model
model = load_model("/content/drive/My Drive/keras_facenet/keras-facenet/model/facenet_keras.h5" )

In [ ]:
model.summary()

Model: "inception_resnet_v1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
________________________________________________________________________________

In [ ]:
from keras.layers import Flatten,Dense
from keras import Model

In [ ]:
last_layer = model.get_layer('Bottleneck_BatchNorm').output
outcome = Dense(217, activation='softmax', name='Classification_layer')(last_layer)

In [ ]:
model = Model(model.input,outcome)

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
____________________________________________________________________________________________

In [ ]:
layercount = 0
for layer in model.layers:
	layercount = layercount+1

In [ ]:
for l in range(layercount-1):
    model.layers[l].trainable=False

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
____________________________________________________________________________________________

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
def preprocessinput(face):
  face = face.astype('float32')
  # standardization
  mean, std = face.mean(), face.std()
  face = (face-mean)/std
  return face

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagenerator = ImageDataGenerator(preprocessing_function = preprocessinput, horizontal_flip=True)

In [ ]:
class SelfSaver(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if epoch%5==0: 
            self.model.save("model_{}.h5".format(epoch))

In [ ]:
!unzip -q "/content/drive/MyDrive/lfw/drive-download-20210304T032757Z-001.zip"

In [70]:
train_generator = datagenerator.flow_from_directory(
        '/content/drive/MyDrive/lfw/train',
        target_size=(160, 160),
        class_mode='categorical',
        )

validation_generator = datagenerator.flow_from_directory(
        '/content/drive/MyDrive/lfw/val',
        target_size=(160, 160),
        class_mode='categorical',
        )

BS = 32
history = model.fit(
        train_generator,
        validation_data=validation_generator,
        steps_per_epoch=3280// BS,
        validation_steps= 1542 // BS, 
        epochs=30,
        callbacks=[SelfSaver()])





Found 3280 images belonging to 217 classes.
Found 1542 images belonging to 217 classes.
Epoch 1/30
102/102 [==============================] - 1443s 14s/step - loss: 4.6454 - accuracy: 0.1392 - val_loss: 4.1139 - val_accuracy: 0.1888
Epoch 2/30
102/102 [==============================] - 160s 2s/step - loss: 3.6478 - accuracy: 0.2620 - val_loss: 3.4878 - val_accuracy: 0.2943
Epoch 3/30
102/102 [==============================] - 159s 2s/step - loss: 3.0950 - accuracy: 0.3611 - val_loss: 3.0922 - val_accuracy: 0.3620
Epoch 4/30
102/102 [==============================] - 159s 2s/step - loss: 2.7062 - accuracy: 0.4412 - val_loss: 2.8003 - val_accuracy: 0.4102
Epoch 5/30
102/102 [==============================] - 159s 2s/step - loss: 2.3953 - accuracy: 0.5068 - val_loss: 2.5778 - val_accuracy: 0.4551
Epoch 6/30
102/102 [==============================] - 160s 2s/step - loss: 2.1804 - accuracy: 0.5508 - val_loss: 2.3984 - val_accuracy: 0.4876
Epoch 7/30
102/102 [==============================] 

In [71]:
model.save('facereco.h5')